In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Load data
df = pd.read_csv(r"C:\Users\vishn\OneDrive\Documents\Machine Learning\Vishnu_phd.csv")
X = df.drop(columns=["FoS", "SeismicFoS"]).values
y = df["FoS"].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create DataLoader
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)

# Define ANN
class ANNModel(nn.Module):
    def __init__(self, input_dim):
        super(ANNModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.layers(x)

# Train ANN
model = ANNModel(X.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(300):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    train_pred = model(X_train_tensor).numpy()
    test_pred = model(X_test_tensor).numpy()

print("R2 (train):", r2_score(y_train, train_pred))
print("R2 (test):", r2_score(y_test, test_pred))

print("RMSE (train):", mean_squared_error(y_train, train_pred, squared=False))
print("RMSE (test): ", mean_squared_error(y_test, test_pred, squared=False))

print("MAE (train):", mean_absolute_error(y_train, train_pred))
print("MAE (test): ", mean_absolute_error(y_test, test_pred))


R2 (train): 0.9989221780101966
R2 (test): 0.8864285208495825
RMSE (train): 0.03785507942579183
RMSE (test):  0.42872663960398105
MAE (train): 0.025067336118673983
MAE (test):  0.29165300335546934


C:\Users\vishn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\vishn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [2]:
import matplotlib.pyplot as plt
import os

# 📁 Save directory
save_dir = r"C:\Users\vishn\OneDrive\Documents\Machine Learning\ANN_simple"
os.makedirs(save_dir, exist_ok=True)

# ------------------- Actual vs Predicted (Train) -------------------
plt.figure(figsize=(6,4))
plt.scatter(y_train, train_pred, alpha=0.6, color='blue', label='Train')
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--')
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Actual vs Predicted - Train")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "actual_vs_predicted_train.png"))
plt.close()

# ------------------- Actual vs Predicted (Test) -------------------
plt.figure(figsize=(6,4))
plt.scatter(y_test, test_pred, alpha=0.6, color='green', label='Test')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Actual vs Predicted - Test")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "actual_vs_predicted_test.png"))
plt.close()

# ------------------- Residuals (Train) -------------------
residuals_train = y_train - train_pred.flatten()
plt.figure(figsize=(6,4))
plt.scatter(range(len(residuals_train)), residuals_train, color='blue', alpha=0.5)
plt.axhline(0, linestyle='--', color='red')
plt.title("Residuals - Train")
plt.ylabel("Error")
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "residuals_train.png"))
plt.close()

# ------------------- Residuals (Test) -------------------
residuals_test = y_test - test_pred.flatten()
plt.figure(figsize=(6,4))
plt.scatter(range(len(residuals_test)), residuals_test, color='green', alpha=0.5)
plt.axhline(0, linestyle='--', color='red')
plt.title("Residuals - Test")
plt.ylabel("Error")
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "residuals_test.png"))
plt.close()

# ------------------- Loss Curve (if tracked) -------------------
if 'loss_history' in globals() and len(loss_history) > 0:
    plt.figure(figsize=(6,4))
    plt.plot(loss_history, color='purple')
    plt.xlabel("Epoch")
    plt.ylabel("Loss (MSE)")
    plt.title("Loss Curve over Epochs")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, "loss_curve.png"))
    plt.close()
